# Video Actor Synchroncy and Causality (VASC)
## RAEng: Measuring Responsive Caregiving Project
### Caspar Addyman, 2020
### https://github.com/infantlab/VASC

# Step 2: Reorganise the OpenPose JSON wire frame data

This script uses output from [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) human figure recognition neural network to create labeled wireframes for each figure in each frame of a video. 


The `write_json flag` saves the people pose data using a custom JSON writer. Each JSON file has a set of coordinates and confidence scores for each person identified in the frame. For a given person there is:

> An array pose_keypoints_2d containing the body part locations and detection confidence formatted as x1,y1,c1,x2,y2,c2,.... The coordinates x and y can be normalized to the range [0,1], [-1,1], [0, source size], [0, output size], etc., depending on the flag keypoint_scale (see flag for more information), while c is the confidence score in the range [0,1].

<img src="keypoints_pose_25.png" alt="BODY-25 mapping" width="240"/>

### 2.1 - import modules and initialise variables

In [1]:
import os                #operating system functions
import math              #simple math
import glob              #file listing
import json              #importing and exporting json files 
import cv2               #computervision toolkit
import numpy as np       #tools for numerical data
import pandas as pd      
import logging
import ipywidgets as widgets  #let's us add buttons and sliders to this page.
from ipycanvas import Canvas


import matplotlib.pyplot as plt
%matplotlib inline

import vasc #a module of our own functions (found in vasc.py in this folder)

#turn on debugging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
%pdb on   

Automatic pdb calling has been turned ON


#### 2.1.1 - anonymise the videos?

Setting the `anon` flag to 

* `True` - we will not display just the wireframes on black backround without the underlying images from the video. 
* `False` - we will *attempt to* draw video images - If videos are not available we fall back to anonymous mode 

In [2]:
anon = False

### 2.2 - Where are the data?

This routine only needs to know where to find the processed data  and what are the base names. The summary information is listed in the `videos.json` file we created. The raw numerical data is in `allframedata.npz`.

In [3]:
# where's the project folder?
jupwd =  os.getcwd() + "\\"
projectpath = os.getcwd() + "\\..\\lookit\\"

# locations of videos and output
videos_in = projectpath 
videos_out   = projectpath + "out"
videos_out_openpose   = videos_out + "\\openpose"
videos_out_timeseries = videos_out + "\\timeseries"
videos_out_analyses   = videos_out + "\\analyses"

print(videos_out_openpose)
print(videos_out_timeseries)
print(videos_out_analyses)

C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\out\openpose
C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\out\timeseries
C:\Users\cas\OneDrive - Goldsmiths College\Projects\Measuring Responsive Caregiving\VASC\..\lookit\out\analyses


In [4]:
#retrieve the list of base names of processed videos.
with open(videos_out + '\\videos.json') as json_file:
    videos = json.load(json_file)

In [4]:
#optional - check the json
for vid in videos:  
    print(vid)
    print(videos[vid])

lookit.01
{'fullname': 'lookit.01.mp4', 'fullpath': 'C:\\Users\\Cas\\OneDrive - Goldsmiths College\\Projects\\Measuring Responsive Caregiving\\lookit\\lookit.01.mp4', 'index': 0, 'format': '.mp4', 'openpose': {'exitcode': 0, 'when': '2020-02-24T10:33:45.063115', 'out': 'C:\\Users\\Cas\\OneDrive - Goldsmiths College\\Projects\\Measuring Responsive Caregiving\\lookit\\out\\openpose\\lookit.01_output.avi'}, 'height': 480, 'width': 640, 'fps': 15, 'n_frames': 162, 'frames': 162, 'maxpeople': 4, 'start': 0, 'end': 161}
lookit.02
{'fullname': 'lookit.02.mp4', 'fullpath': 'C:\\Users\\Cas\\OneDrive - Goldsmiths College\\Projects\\Measuring Responsive Caregiving\\lookit\\lookit.02.mp4', 'index': 1, 'format': '.mp4', 'openpose': {'exitcode': 0, 'when': '2020-02-24T10:33:56.472137', 'out': 'C:\\Users\\Cas\\OneDrive - Goldsmiths College\\Projects\\Measuring Responsive Caregiving\\lookit\\out\\openpose\\lookit.02_output.avi'}, 'height': 480, 'width': 640, 'fps': 19, 'n_frames': 162, 'frames': 161, 

In [5]:
#can reload the values without recomputing
reloaded = np.load('allframedata.npz')
keypoints_original = reloaded["keypoints_array"]
keypoints_array = np.copy(keypoints_original)

In [6]:
keypoints_array.shape

(10, 412, 4, 75)

### Step 2.3 Clean the data

We now have an numpy array called `keypoints_array` containing all the openpose numbers for all videos. Now we need to do some cleaning of the data. We provide set of tools to do this. There are several tasks we need to do.

1. Tag the first and last frames of interest. 
2. Tag the adult & infant in first frame of interest. So both individuals should be in first frame.
3. Try to automatically tag then in subsequent frames.
4. Manually fix anything the automatic process gets wrong.
5. Exclude other detected people (3rd parties & false positives)


### Step 2.3.1: Where does the interesting data start and end?

For many videos, the period of interest might start (and end) some time into the video. For now we are using the whole video .
TODO  - We wiil give the user the opportunity to set these.


In [7]:
#let's loop through the processed list and set and startframe and endframe for each video
# for the moment we'll just use the full video.
# TODO - we will let the use specify this per video 
for vid in videos:
    videos[vid]["start"] = 0
    videos[vid]["end"] = videos[vid]["frames"] -1

### Step 2.3.2: Tag the actors of interest at start

We want to know which person is the adult and which is the infant in the first frame. We want the child to be Person 0 and Adult to be Person 1. The buttons below provide the choice to swap data series so that this is correct.

For example, if child data starts in series 3, we pick 3 in the drop down list next to button `Swap to child (0)` and then press the button. This will swap these two series.

This function operates beyond the current frame so it's possible to use it multiple times if data jumps around. However, there is a short cut for part of this process..

### Step 2.3.3: Track actors frame by frame

At present OpenPose doesn't track individuals from one frame to the next (I believe they are working on this). It just labels each person in each frame. This means that Person 1 in frame 1 might become Person 2 by frame 100. Here we provide some tools that automatically trying to guess who is who. This is tricky so we also ask for human input. 

Once the child and adult data start in series 0 and 1 respectively, press the `Auto fix` button.

If this leaves a few errors we can move slider to affected frame and use the swap series function to manually correct.

### Step 2.3.4: Exclude other people

Finally we can delete any people in background or false positives (ghosts) detected by OpenPose. We simply set these to zero.

In [8]:
def swapSeries(keypoints_array,vidx,pers1,pers2,start,end):
    """helper function for swapping sections of time series. This is useful because openpose isn't
       consistent in labelling people so we need to rearrange things.
    Args:
        keypoints_array: all the data.
        vidx: which video? - specifies first dimension of array
        pers1: which people to swap 1
        pers2: which people to swap 2
        start: where in time series do we start? (TODO can be blank - start at beginning)
        end: where in time series do we end? (TODO can be blank - to end)
    Returns:
        a rearranged keypoints_array
    """
    temp = np.copy(keypoints_array[vidx,start:end,pers1,:])  #temporary copy pers1 
    keypoints_array[vidx,start:end,pers1,:] = keypoints_array[vidx,start:end,pers2,:] #pers2 to pers 1
    keypoints_array[vidx,start:end,pers2,:] = temp
    
    return keypoints_array
        
def deleteSeries(keypoints_array,vidx,pers,start,end):
    """helper function for deleting time series that aren't parent or child.
    Args:
        keypoints_array: all the data.
        vidx: which video? - specifies first dimension of array
        pers: which person to delete
        start: where in time series do we start? (TODO can be blank - start at beginning)
        end: where in time series do we end? (TODO can be blank - to end)
    Returns:
        a rearranged keypoints_array
    """
    #simply set all these values to zero
    keypoints_array[vidx,start:end,pers,:] = 0
    #TODO - update the corresponding json file.
    return keypoints_array
    
def fixpeopleSeries(keypoints_array,vidx,people, start, end):
    """ openpose isn't consistent in labelling people (person 1 could be labeled pers 2 in next frame).
    So we go through frame by frame and label people in new frame with index of nearest person from previous frame. This *should* fix things.
    I have horrible feeling it will get messed up by missing data. Let's find out..
    Args:
        keypoints_array: all the data.
        vidx: which video? - specifies first dimension of array
        people: list of all people we are comparing. 
        start: where in time series do we start? (TODO can be blank - start at beginning)
        end: where in time series do we end? (TODO can be blank - to end)
    Returns:
        a rearranged keypoints_array
    """
    #This may get messy!
    for f in range(start,end-1):
        print ("frame ", '{:4d}'.format(f))
        # we loop through all pairs of people
        for p1 in range(len(people)-1):
            r1 = people[p1] #what row is this?
            whosleft = range(p1,len(people))
            delta = {}
            for p2 in whosleft: 
                r2 = people[p2] #what row is this?
                delta[p2] = vasc.diffKeypoints(keypoints_array[vidx,f,p1,:],keypoints_array[vidx,f+1,p2,:],vasc.xys)
                print(p1,p2,np.nansum(delta[p2]),any(np.isfinite(delta[p2])))
            #this next line finds the minimum & non-null value
            key_min = min(np.nansum(delta.keys()), key=(lambda k: delta[k]))
            print (key_min)
            if key_min != p1:
                #swap the rest of series between these two 
                keypoints_array = swapSeries(keypoints_array,vidx,p1,key_min,f+1,end)


In [9]:
canvas = Canvas(width=800, height=600)

vidlist = [] #useful for displaying
for vid in videos:  
    vidlist.append(vid)

pickvid = widgets.Dropdown(
    options= vidlist,
    value= vidlist[0],
    description='Select video:'
)

#pressing this button swaps one set of data to be index 0 - default for child.
button_swapchild = widgets.Button(description="Swap to child (0)")
child = widgets.Dropdown(
    options = list(range(4)),
    value= 0,
    description='Set: '
)
babybox = widgets.HBox([button_swapchild, child])
adult = widgets.Dropdown(
    options = list(range(4)),
    value= 1,
    description='Set: '
)
button_swapadult = widgets.Button(description="Swap to adult (1)")
adultbox = widgets.HBox([button_swapadult,adult])

button_remove = widgets.Button(description="Remove these data")
remove = widgets.Dropdown(
    options = list(range(4)),
    value= 2,
    description='Set: '
)
removebox = widgets.HBox([button_remove,remove])


slider = widgets.IntSlider(
    value=0,
    min=0,
    max=161,
    step=1,
    description='Frame:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

button_update = widgets.Button(description="Redraw")
button_fixseries = widgets.Button(description="Auto fix")
button_reset_one = widgets.Button(description="Reset this video")
button_reset_all = widgets.Button(description="Reset all")
buttonbox = widgets.HBox([button_update,button_fixseries,button_reset_one,button_reset_all])
output = widgets.Output()

def pickvid_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        updateAll(True)

def slider_change(slider):
    updateAll(False)

def on_button_clicked(output):
    logging.info('button_update_all clicked')
    updateAll(True)

def on_reset_all(output):
    global keypoints_array
    logging.info('button_reset_all clicked')
    keypoints_array = np.copy(keypoints_original)
    updateAll(True)

def on_fixseries(output):
    global keypoints_array
    logging.info('on_fixseries')
    v = videos[pickvid.value]["index"]
    end  = videos[pickvid.value]["end"]
    fixpeopleSeries(keypoints_array,v,[0,1],slider.value, end)
    updateAll(True)

def on_deleteseries(output):
    global keypoints_array
    logging.info('on_fixseries')
    v = videos[pickvid.value]["index"]
    end  = videos[pickvid.value]["end"]
    deleteSeries(keypoints_array,v,remove.value,slider.value, end)
    updateAll(True)
    
def on_swapchild(output):
    global keypoints_array
    logging.info('on_swapchild')
    v = videos[pickvid.value]["index"]
    end  = videos[pickvid.value]["end"]
    swapSeries(keypoints_array,v,0,child.value,slider.value,end)
    updateAll(True)

def on_swapadult(output):
    global keypoints_array
    logging.info('on_swapadult')
    v = videos[pickvid.value]["index"]
    end  = int(videos[pickvid.value]["end"])
    swapSeries(keypoints_array,v,1,adult.value,slider.value,end)
    updateAll(True)


slider.observe(slider_change, 'value')
pickvid.observe(pickvid_change, 'value')
button_swapchild.on_click(on_swapchild)
button_swapadult.on_click(on_swapadult)
button_fixseries.on_click(on_fixseries)
button_remove.on_click(on_deleteseries)
button_update.on_click(on_button_clicked)
button_reset_all.on_click(on_reset_all)

##functions to draw complicated stuff..
def drawOneFrame(frameNum):
    vid = pickvid.value
    v = videos[vid]["index"]  # which subarray of data do we need?
    vidpath = videos_in + "\\" + videos[vid]["fullname"]
    frame = vasc.getframeimage(vidpath,frameNum) 
    vasc.drawPoints(frame,keypoints_array[v,frameNum,:,:],videos[vid]["maxpeople"])
    vasc.drawLines(frame,keypoints_array[v,frameNum,:,:],videos[vid]["maxpeople"])
    vasc.drawBodyCG(frame,keypoints_array[v,frameNum,:,:],videos[vid]["maxpeople"])
    #send the image to the canvas
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hiddencanvas = Canvas(width=img.shape[1], height=img.shape[0])
    hiddencanvas.put_image_data(img, 0, 0)
    canvas.draw_image(hiddencanvas,0,0,canvas.width,canvas.height)
    canvas.restore()
    
def drawMovementGraph(vid, points, frame = 0, average = True):
    v = videos[vid]["index"]
    N = videos[vid]["frames"]
    t = np.zeros([N,1])
    t[:,0]= list(range(N))

    ceegees = np.zeros([N,videos[vid]["maxpeople"]])

    for frameNum in range(N):
        for p in range(videos[vid]["maxpeople"]):
            personkeypoints = keypoints_array[v,frameNum,p,:]
            avx = vasc.averagePoint(personkeypoints,vasc.xs)
            if (avx > 0):
                ceegees[frameNum,p] = avx
            else:
                ceegees[frameNum,p] = None

    plt.figure(figsize=(12, 4))
    plt.plot(t,ceegees)
    logging.info("frame = " + str(frame))
    plt.axvline(x=frame,c='tab:cyan')
    plt.title('Horizontal movement of people (average) over time.)')
    plt.legend([0, 1, 2, 3])
    plt.show()

def updateAll(forceUpdate = False):
    output.clear_output()
    if forceUpdate:
        slider.value = 0
        slider.max = videos[pickvid.value]["end"]
    with output:
        display(canvas,pickvid, babybox,adultbox,removebox, slider, buttonbox)  
        drawOneFrame(slider.value)
        drawMovementGraph(pickvid.value,vasc.xs,slider.value,True)
    
#draw everything for first time
updateAll(True)
output

Output()

### Step 2.4: TODO - Correct for camera motion?

Some video sets the camera is not fixed. Any camera movements will cause perfectly correlated movements in the pair of signals. We need to decide what (if anything) to do about this. (Not yet implemented.)

### Step 2.5: TODO - Interpolate missing data

There are still likely to be gaps. We need to decide what to do about those.  (Not yet implemented.) 

### Step 2.6: TODO - Exclude whole video

Some time the data will look too bad to use. In which case, we need to completely remove this whole set. (Not yet implemented.) 

In [ ]:
(Not yet implemented.) 

## Step 2.6: Save the data!

Saving the data at this stage so we don't have to repeat these steps again if we reorganise or reanalyse the data.

We create a compressed NumPy array `cleandata.npz` containing the person location data for all the videos. 

We also update the `videos.json` file with more info about the videos. 

In [24]:
#update the json file in the video out directory
with open(videos_out + '\\videos.json', 'w') as outfile:
    json.dump(videos, outfile)

# in the time series folder we save the data file. 
#in a compressed format as it has a lot of empty values
np.savez_compressed(videos_out_timeseries + '\\cleandata.npz', keypoints_array=keypoints_array)



#### That's it. 

Now go onto [Step 3 - Analyse the data](Step3.AnalyseData.scipy)